## Q1. Embedding the query

In [ ]:
from fastembed import TextEmbedding
import numpy as np
from qdrant_client import QdrantClient, models
from qdrant_client.http import models


In [ ]:
query = "I just discovered the course. Can I join now?"

In [ ]:
embedding_model = TextEmbedding('jinaai/jina-embeddings-v2-small-en')

In [ ]:
embeddings_generator = embedding_model.embed(query)
embeddings_list = list(embeddings_generator)

In [ ]:
round(np.min(embeddings_list[0]),3)

**Checking if its normalized**

In [ ]:
import numpy as np
np.linalg.norm(embeddings_list[0])

In [ ]:
embeddings_list[0].dot(embeddings_list[0])

## Q2. Cosine similarity with another vector

In [ ]:
doc = 'Can I still join the course after the start date?'

**Embedding the doc text**

In [ ]:
embeddings_generator = embedding_model.embed(doc)
embeddings_doc = list(embeddings_generator)

**checking the cosine similarity**

In [ ]:
round(embeddings_doc[0].dot(embeddings_list[0]),2)

## Q3. Ranking by cosine

In [ ]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

**Create collection**

In [ ]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [ ]:
# Define the collection name
collection_name = "zoomcamp-Q3"
EMBEDDING_DIMENSIONALITY = 512

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

## Create, Embed & Insert Points into the Collection

**connecting to  client**

In [ ]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [ ]:
embedding_model = TextEmbedding('jinaai/jina-embeddings-v2-small-en')

In [ ]:
points = []
id = 0

for doc in documents:
    embedding = list(embedding_model.embed(doc['text']))[0]
    
    point = models.PointStruct(
        id=id,
        vector=embedding, #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": doc['course']
        } #save all needed metadata fields
            )
    points.append(point)
    id += 1

In [ ]:
client.upsert(
    collection_name=collection_name,
    points=points
)

In [ ]:
cosine_scores = []

for point in points:
    vector = np.array(point.vector)
    norm_value = np.linalg.norm(vector)
    cosine = vector.dot(embeddings_list[0])
    cosine_scores.append((point.id, cosine, norm_value))

# Sort by cosine similarity in descending order
cosine_scores.sort(key=lambda x: x[1], reverse=True)

# Print sorted results
for id, cosine, norm in cosine_scores:
    print(f"ID: {id}, Cosine: {cosine}, norm: {norm}")


## Q4. Ranking by cosine, version two

In [ ]:
def merge_doc(docs):
    for doc in docs:
        full_text = doc['question'] + ' ' + doc['text']
        doc['full_text'] = full_text
    return docs

In [ ]:
documets2 = documents.copy()

In [ ]:
document_new = merge_doc(documets2)

**Create collection**

In [ ]:
# Define the collection name
collection_name = "zoomcamp-Q4"
EMBEDDING_DIMENSIONALITY = 512

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

**Model embedding**

In [ ]:
points = []
id = 0

for doc in document_new:
    embedding = list(embedding_model.embed(doc['full_text']))[0]
    
    point = models.PointStruct(
        id=id,
        vector=embedding, #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "full_text": doc['full_text'],
            "section": doc['section'],
            "course": doc['course']
        } #save all needed metadata fields
            )
    points.append(point)
    id += 1

In [ ]:
client.upsert(
    collection_name=collection_name,
    points=points
)

In [ ]:
cosine_scores = []

for point in points:
    vector = np.array(point.vector)
    norm_value = np.linalg.norm(vector)
    cosine = vector.dot(embeddings_list[0])
    cosine_scores.append((point.id, cosine, norm_value))

# Sort by cosine similarity in descending order
cosine_scores.sort(key=lambda x: x[1], reverse=True)

# Print sorted results
for id, cosine, norm in cosine_scores:
    print(f"ID: {id}, Cosine: {cosine}, norm: {norm}")


## Q5. Selecting the embedding model

In [ ]:
sorted_models = sorted(TextEmbedding.list_supported_models(), key=lambda x: x['dim'])
#sorted_models[0]['dim']
sorted_models[0]

## Q6. Indexing with qdrant (2 points)

In [ ]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [ ]:
documents

In [ ]:
def merge_text(docs):
    for doc in docs:
        text = doc['question'] + ' ' + doc['text']
        doc['text'] = text
    return docs

In [ ]:
document_x = merge_text(documents)

In [ ]:
document_x

In [ ]:
embedding_model = TextEmbedding('BAAI/bge-small-en')

In [ ]:
# Define the collection name
collection_name = "zoomcamp-Q6"
EMBEDDING_DIMENSIONALITY = 512

In [ ]:
# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

In [ ]:
points = []
id = 0

for doc in document_x:
    embedding = list(embedding_model.embed(doc['text']))[0]
    
    point = models.PointStruct(
        id=id,
        vector=embedding, #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": doc['course']
        } #save all needed metadata fields
            )
    points.append(point)
    id += 1

In [ ]:
client.upsert(
    collection_name=collection_name,
    points=points
)

In [ ]:
embeddings_generator = embedding_model.embed(query)
embeddings_list = list(embeddings_generator)

In [238]:
cosine_scores = []

for point in points:
    vector = np.array(point.vector)
    norm_value = np.linalg.norm(vector)
    cosine = vector.dot(embeddings_list[0])
    cosine_scores.append((point.id, cosine, norm_value))

# Sort by cosine similarity in descending order
cosine_scores.sort(key=lambda x: x[1], reverse=True)

# Print sorted results
for id, cosine, norm in cosine_scores:
    print(f"ID: {id}, Cosine: {cosine}, norm: {norm}")


ID: 14, Cosine: 0.8703172355728512, norm: 0.9999999743331559
ID: 6, Cosine: 0.8691884656117479, norm: 1.0000000205261674
ID: 13, Cosine: 0.8683311072522578, norm: 0.9999999433836532
ID: 293, Cosine: 0.8576105596771215, norm: 0.9999999671558005
ID: 17, Cosine: 0.8571540389939137, norm: 1.0000000732477647
ID: 231, Cosine: 0.8568166145986099, norm: 0.9999999862485074
ID: 262, Cosine: 0.8544782997176332, norm: 0.999999945854573
ID: 9, Cosine: 0.8455939052447949, norm: 0.9999999960299701
ID: 5, Cosine: 0.8414748014456602, norm: 1.0000000138568965
ID: 18, Cosine: 0.8347909904289127, norm: 1.0000000356151506
ID: 11, Cosine: 0.8321303284298623, norm: 1.0000000501770085
ID: 4, Cosine: 0.8234946554174352, norm: 1.000000007681959
ID: 15, Cosine: 0.8224412347165199, norm: 1.0000000032869332
ID: 20, Cosine: 0.8210680242218888, norm: 1.0000000378380816
ID: 1, Cosine: 0.8197733627970245, norm: 1.0000000634668407
ID: 50, Cosine: 0.819571144233962, norm: 0.9999999197665019
ID: 62, Cosine: 0.81823931962